In [189]:
from PIL import Image 
from IPython.display import display 
import numpy as np
import random
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

In [229]:
# Import trait files

traits_file = pd.read_csv("Traits-Table.csv", header=0)
exception_file = pd.read_csv("Exceptions-Table.csv", header=0)
traits_file

,trait_groups,trait_variations,file_name,quant,weights,old_name,new_name,Unnamed: 7
0,Background,Lines,Lines Bg,588.0,0.0588,Lines Bg,NaN,NaN
1,Background,Green,Bg1,588.0,0.0588,Bg1,NaN,NaN
2,Background,Pink,Bg2,588.0,0.0588,Bg2,NaN,NaN
3,Background,Light Pink,Bg3,588.0,0.0588,Bg3,NaN,NaN
4,Background,Black,Black Bg,588.0,0.0588,Black Bg,NaN,NaN
...,...,...,...,...,...,...,...,...
224,Clothing,Royal Coat,19 Cloth,100.0,0.0100,19 Cloth,NaN,NaN
225,Clothing,None,None,400.0,0.0400,None,NaN,NaN
226,Clothing2,Royal Coat,19 Cloth,NaN,0.0000,19 Cloth,NaN,NaN
227,Ancestry,Demigod,demi,2000.0,0.2000,demi,NaN,NaN


In [230]:
# Create lists of trait variations for each group

background = traits_file["trait_variations"][traits_file["trait_groups"] == "Background"].values.tolist()
skin = traits_file["trait_variations"][traits_file["trait_groups"] == "Skin"].values.tolist()
clothing = traits_file["trait_variations"][traits_file["trait_groups"] == "Clothing"].values.tolist()
glasses = traits_file["trait_variations"][traits_file["trait_groups"] == "Glasses"].values.tolist()
hand = traits_file["trait_variations"][traits_file["trait_groups"] == "Hands"].values.tolist()
hat = traits_file["trait_variations"][traits_file["trait_groups"] == "Hat"].values.tolist()
mask = traits_file["trait_variations"][traits_file["trait_groups"] == "Mask"].values.tolist()
several = traits_file["trait_variations"][traits_file["trait_groups"] == "Several"].values.tolist()
ancestry = traits_file["trait_variations"][traits_file["trait_groups"] == "Ancestry"].values.tolist()


# List to be used in layer sequences only

glasses2 = traits_file["trait_variations"][traits_file["trait_groups"] == "Glasses2"].values.tolist()
glasses3 = traits_file["trait_variations"][traits_file["trait_groups"] == "Glasses3"].values.tolist()
glasses4 = traits_file["trait_variations"][traits_file["trait_groups"] == "Glasses4"].values.tolist()
several2 = traits_file["trait_variations"][traits_file["trait_groups"] == "Several2"].values.tolist()
several1 = traits_file["trait_variations"][traits_file["trait_groups"] == "Several1"].values.tolist()
several3 = traits_file["trait_variations"][traits_file["trait_groups"] == "Several3"].values.tolist()
hat2 = traits_file["trait_variations"][traits_file["trait_groups"] == "Hat2"].values.tolist()
hat3 = traits_file["trait_variations"][traits_file["trait_groups"] == "Hat3"].values.tolist()
hat4 = traits_file["trait_variations"][traits_file["trait_groups"] == "Hat4"].values.tolist()
hat5 = traits_file["trait_variations"][traits_file["trait_groups"] == "Hat5"].values.tolist()
beard1 = traits_file["trait_variations"][traits_file["trait_groups"] == "Beard1"].values.tolist()
beard2 = traits_file["trait_variations"][traits_file["trait_groups"] == "Beard2"].values.tolist()
clothing2 = traits_file["trait_variations"][traits_file["trait_groups"] == "Clothing2"].values.tolist()

In [231]:
# Create weights for each trait variation

background_weights = traits_file["weights"][traits_file["trait_groups"] == "Background"].values.tolist()
skin_weights = traits_file["weights"][traits_file["trait_groups"] == "Skin"].values.tolist()
clothing_weights = traits_file["weights"][traits_file["trait_groups"] == "Clothing"].values.tolist()
glasses_weights = traits_file["weights"][traits_file["trait_groups"] == "Glasses"].values.tolist()
hand_weights = traits_file["weights"][traits_file["trait_groups"] == "Hands"].values.tolist()
hat_weights = traits_file["weights"][traits_file["trait_groups"] == "Hat"].values.tolist()
mask_weights = traits_file["weights"][traits_file["trait_groups"] == "Mask"].values.tolist()
several_weights = traits_file["weights"][traits_file["trait_groups"] == "Several"].values.tolist()
ancestry_weights = traits_file["weights"][traits_file["trait_groups"] == "Ancestry"].values.tolist()

In [232]:
glasses_weights

[0.065, 0.01, 0.045, 0.055, 0.045, 0.08, 0.03, 0.04, 0.05, 0.58]

In [233]:
# Import file names (if there is more than on trait with the same name, this has to be adjusted)

file_names = traits_file[["trait_variations", "file_name"]]
file_names_list = file_names.to_dict('split')["data"]
file_names_dict = {k[0]: k[1:][0] for k in file_names_list}

In [244]:
## Generate Traits

TOTAL_IMAGES = 7500  # Number of random unique images we want to generate

all_images = [] 
all_images_no_bg = []

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} 
    new_image_no_bg = {} #

    # For each trait category, select a random trait based on the weightings 
    new_image ["background"] = random.choices(background, background_weights)[0]
    new_image ["skin"] = random.choices(skin, skin_weights)[0]
    new_image ["clothing"] = random.choices(clothing, clothing_weights)[0]
    new_image ["glasses"] = random.choices(glasses, glasses_weights)[0]
    new_image ["hand"] = random.choices(hand, hand_weights)[0]
    new_image ["hat"] = random.choices(hat, hat_weights)[0]
    new_image ["mask"] = random.choices(mask, mask_weights)[0]
    new_image ["several"] = random.choices(several, several_weights)[0]
    
  
    # Treat exceptions
    
    new_image_list = list(new_image.values())
    for i in new_image_list:
        exception_list = list(exception_file[exception_file["key_trait"] == i]["Exception"])
        if len((set(new_image_list) & set(exception_list))) != 0:
            return create_new_image()



   
    # Removing background, eyes, mouth from uniqueness comparison
    new_image_no_bg["skin"] = new_image ["skin"]
    new_image_no_bg["clothing"] = new_image ["clothing"]
    new_image_no_bg["glasses"] = new_image ["glasses"]
    new_image_no_bg["hat"] = new_image ["hat"]
    
    

    
    if new_image_no_bg in all_images_no_bg:
        return create_new_image()
    else:
        return new_image, new_image_no_bg
    
    
    #if new_image in all_images:
    #    return create_new_image()
    #else:
    #    return new_image
                                              
   
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image[0])
    all_images_no_bg.append(new_trait_image[1])


    



In [245]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))




Are all images unique? True


In [246]:
# Returns true if all images excluding background and other qualitative traits are unique
def all_images_unique(all_images_no_bg):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images_no_bg)
#
print("Are all images unique regardless?", all_images_unique(all_images_no_bg))






Are all images unique regardless? True


In [247]:
## Export file for review

# Add temporary token id:

i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

# Create pandas



df_tb_reviewed = pd.DataFrame.from_dict(all_images)
df_tb_reviewed = df_tb_reviewed.set_index('tokenId')

df_tb_reviewed.to_csv("file_tb_reviewed.csv")




In [248]:
## Import reviewed file

#df_reviewed = pd.read_csv("file_reviewed.csv", header=0)
#all_images_rev = df_reviewed.to_dict(orient='records')
#len(all_images_rev)

In [249]:
import copy

# ONLY RUN THIS IF SKIPPING FILE REVIEW

all_images_rev = copy.deepcopy(all_images)

In [250]:
# Add token Id to each image
i = 0
for item in all_images_rev:
    item["tokenId"] = i
    i = i + 1
#    if i<=650: 
#        item["ancestry"] = "god"
#    elif 1500>=i>650:
#        item["ancestry"] = "demi"
#    elif 3000>=i>1500:
#        item["ancestry"] = "king"
#    elif 5000>=i>3000:
#        item["ancestry"] = "prince"
#    elif 10000>=i>5000:
#        item["ancestry"] = "duke"
        
        

In [251]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0
    
skin_count = {}
for item in skin:
    skin_count[item] = 0
    
    
clothing_count = {}
for item in clothing:
    clothing_count[item] = 0

glasses_count = {}
for item in glasses:
    glasses_count[item] = 0
    
    
hand_count = {}
for item in hand:
    hand_count[item] = 0
    
    
hat_count = {}
for item in hat:
    hat_count[item] = 0
    
    
    
mask_count = {}
for item in mask:
    mask_count[item] = 0
    
several_count = {}
for item in several:
    several_count[item] = 0
    
ancestry_count = {}
for item in ancestry:
    ancestry_count[item] = 0
    
    

for image in all_images_rev:
    background_count[image["background"]] += 1
    skin_count[image["skin"]] += 1
    clothing_count[image["clothing"]] += 1
    glasses_count[image["glasses"]] += 1
    hand_count[image["hand"]] += 1
    hat_count[image["hat"]] += 1
    mask_count[image["mask"]] += 1
    several_count[image["several"]] += 1
    

In [252]:
# Create Pandas df for images
df_tokens = pd.DataFrame.from_dict(all_images_rev)
df_tokens = df_tokens.set_index('tokenId')

df_tokens.to_csv("file_tokens.csv")

In [253]:
# Create Pandas df for rarity

df_rarity_1 = pd.DataFrame.from_dict(OrderedDict(sorted(background_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_1['trait_group']="background"

df_rarity_2 = pd.DataFrame.from_dict(OrderedDict(sorted(skin_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_2['trait_group']="skin"

df_rarity_3 = pd.DataFrame.from_dict(OrderedDict(sorted(clothing_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_3['trait_group']="clothing"

df_rarity_4 = pd.DataFrame.from_dict(OrderedDict(sorted(glasses_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_4['trait_group']="glasses"

df_rarity_6 = pd.DataFrame.from_dict(OrderedDict(sorted(hand_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_6['trait_group']="hand"

df_rarity_7 = pd.DataFrame.from_dict(OrderedDict(sorted(hat_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_7['trait_group']="hat"


df_rarity_8 = pd.DataFrame.from_dict(OrderedDict(sorted(mask_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_8['trait_group']="mask"

df_rarity_9 = pd.DataFrame.from_dict(OrderedDict(sorted(several_count.items(), key=lambda t: t[1])), orient='index', columns=["count"])
df_rarity_9['trait_group']="several"


df_rarity = pd.concat([df_rarity_1, df_rarity_2, df_rarity_3, df_rarity_4, df_rarity_6, df_rarity_7, df_rarity_8, df_rarity_9])

df_rarity.to_csv("file_rarity.csv")

In [ ]:
all_images_rev_temp = copy.deepcopy(all_images_rev)

#### Generate Images    
for item in all_images_rev_temp:
    
    if item["glasses"] in glasses2:
        item["glasses2"] = item["glasses"]
        item["glasses"] = "None"
        item["glasses3"] = "None"
        item["glasses4"] = "None"
    elif item["glasses"] in glasses3:
        item["glasses3"] = item["glasses"]
        item["glasses"] = "None"
        item["glasses2"] = "None"
        item["glasses4"] = "None"
    elif item["glasses"] in glasses4:
        item["glasses4"] = item["glasses"]
        item["glasses"] = "None"
        item["glasses2"] = "None"
        item["glasses3"] = "None"
    else:
        item["glasses2"] = "None"
        item["glasses3"] = "None"
        item["glasses4"] = "None"
        
        
    if item["mask"] in beard1:
        item["beard1"] = item["mask"]
        item["mask"] = "None"
        item["beard2"] = "None"
    elif item["mask"] in beard2:
        item['beard2'] = item["mask"]
        item["beard1"] = "None"
        item["mask"] = "None"
    else:
        item["beard1"] = "None"
        item["beard2"] = "None"
        
        
        
    if item["several"] in several1:
        item["several1"] = item["several"]
        item["several"] = "None"
        item["several2"] = "None"
        item["several3"] = "None"
    elif item["several"] in several2:
        item["several2"] = item["several"]
        item["several"] = "None"
        item["several1"] = "None"
        item["several3"] = "None"
    elif item["several"] in several3:
        item["several3"] = item["several"]
        item["several"] = "None"
        item["several1"] = "None"
        item["several2"] = "None"
    else:
        item["several1"] = "None"
        item["several2"] = "None"
        item["several3"] = "None"
        
        
    if item["hat"] in hat2:
        item["hat2"] = item["hat"]
        item["hat"] = "None"
        item["hat3"] = "None"
        item["hat4"] = "None"
        item["hat5"] = "None"
    elif item["hat"] in hat3:
        item["hat3"] = item["hat"]
        item["hat"] = "None"
        item["hat2"] = "None"
        item["hat4"] = "None"
        item["hat5"] = "None"
    elif item["hat"] in hat4:
        item["hat4"] = item["hat"]
        item["hat"] = "None"
        item["hat2"] = "None"
        item["hat3"] = "None"
        item["hat5"] = "None"
    elif item["hat"] in hat5:
        item["hat5"] = item["hat"]
        item["hat"] = "None"
        item["hat2"] = "None"
        item["hat3"] = "None"
        item["hat4"] = "None"
    else:
        item["hat2"] = "None"
        item["hat3"] = "None"
        item["hat4"] = "None"
        item["hat5"] = "None"
        
        
    if item["clothing"] in clothing2:
        item["clothing2"] = item["clothing"]
        item["clothing"] = "None"
    else:
        item["clothing2"] = "None"

        

    im1 = Image.open(f'./trait-layers/Background/{file_names_dict[item["background"]]}.png').convert('RGBA')
    im2 = Image.open(f'./trait-layers/Skins/{file_names_dict[item["skin"]]}.png').convert('RGBA')
    im3 = Image.open(f'./trait-layers/Several2/{file_names_dict[item["several2"]]}.png').convert('RGBA')
    im4 = Image.open(f'./trait-layers/Mask/{file_names_dict[item["mask"]]}.png').convert('RGBA')
    im5 = Image.open(f'./trait-layers/Clothing2/{file_names_dict[item["clothing2"]]}.png').convert('RGBA') 
    im6 = Image.open(f'./trait-layers/Hat3/{file_names_dict[item["hat3"]]}.png').convert('RGBA')   
    im7 = Image.open(f'./trait-layers/Glasses2/{file_names_dict[item["glasses2"]]}.png').convert('RGBA')                 
    im8 = Image.open(f'./trait-layers/Hat/{file_names_dict[item["hat"]]}.png').convert('RGBA') 
    im9 = Image.open(f'./trait-layers/Beard2/{file_names_dict[item["beard2"]]}.png').convert('RGBA') 
    im10 = Image.open(f'./trait-layers/Glasses/{file_names_dict[item["glasses"]]}.png').convert('RGBA') 
    im11 = Image.open(f'./trait-layers/Clothing/{file_names_dict[item["clothing"]]}.png').convert('RGBA') 
    im12 = Image.open(f'./trait-layers/Glasses4/{file_names_dict[item["glasses4"]]}.png').convert('RGBA') 
    im13 = Image.open(f'./trait-layers/Hat2/{file_names_dict[item["hat2"]]}.png').convert('RGBA')   
    im14 = Image.open(f'./trait-layers/Beard1/{file_names_dict[item["beard1"]]}.png').convert('RGBA') 
    im15 = Image.open(f'./trait-layers/Hat5/{file_names_dict[item["hat5"]]}.png').convert('RGBA') 
    im16 = Image.open(f'./trait-layers/Glasses3/{file_names_dict[item["glasses3"]]}.png').convert('RGBA') 
    im17 = Image.open(f'./trait-layers/Several1/{file_names_dict[item["several1"]]}.png').convert('RGBA')
    im18 = Image.open(f'./trait-layers/Several3/{file_names_dict[item["several3"]]}.png').convert('RGBA')
    im19 = Image.open(f'./trait-layers/Hat4/{file_names_dict[item["hat4"]]}.png').convert('RGBA')   
    im20 = Image.open(f'./trait-layers/Hands/{file_names_dict[item["hand"]]}.png').convert('RGBA')
    im21 = Image.open(f'./trait-layers/Several/{file_names_dict[item["several"]]}.png').convert('RGBA')
 
    
            
    #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)
    com3 = Image.alpha_composite(com2, im4)
    com4 = Image.alpha_composite(com3, im5)
    com5 = Image.alpha_composite(com4, im6)
    com6 = Image.alpha_composite(com5, im7)
    com7 = Image.alpha_composite(com6, im8)
    com8 = Image.alpha_composite(com7, im9)
    com9 = Image.alpha_composite(com8, im10)
    com10 = Image.alpha_composite(com9, im11)
    com11 = Image.alpha_composite(com10, im12)
    com12 = Image.alpha_composite(com11, im13)
    com13 = Image.alpha_composite(com12, im14)
    com14 = Image.alpha_composite(com13, im15)
    com15 = Image.alpha_composite(com14, im16)
    com16 = Image.alpha_composite(com15, im17)
    com17 = Image.alpha_composite(com16, im18)
    com18 = Image.alpha_composite(com17, im19)
    com19 = Image.alpha_composite(com18, im20)
    com20 = Image.alpha_composite(com19, im21)


    #Convert to RGB
    rgb_im = com20.convert('RGBA')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("../file/images/" + file_name)
    
    
    

In [255]:
#### Generate Metadata for tokens Traits 
METADATA_FILE_NAME = '../XXXX/metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images_rev, outfile, indent=4)

In [256]:
#### Generate Metadata for each Image    


f = open('../XXXX/metadata/all-traits.json',) 
data = json.load(f)


#########

IMAGES_BASE_URI = "XXXXXX"
PROJECT_NAME = "FERRYMEN"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' #' + str(token_id),
        "description": "Weaved by NYX, the Mother Night, Souls are the indivisible essence that defines us. They cross into the shadow lands when we sleep, meditate, daydream and when we die. Corrupted Wyverns and other creatures sweep these lands preying on travelers. There are those who dedicate their existence to guide and protect the crossing. They go by the name FERRYMEN.",
        "attributes": []
    }
    token["attributes"].append(getAttribute("Ferrymen", "Oblivion"))
    token["attributes"].append(getAttribute("Background", i["background"]))
    token["attributes"].append(getAttribute("Tribe", i["skin"]))
    if getAttribute("Glasses", i["glasses"])["value"] !=  "None": #for traits with none
        token["attributes"].append(getAttribute("Glasses", i["glasses"]))
    if getAttribute("Hat", i["hat"])["value"] !=  "None": #for traits with none
        token["attributes"].append(getAttribute("Hat", i["hat"]))
    if getAttribute("Mask", i["mask"])["value"] !=  "None": #for traits with none
        token["attributes"].append(getAttribute("Facial Feature", i["mask"]))
    if getAttribute("Several", i["several"])["value"] !=  "None": #for traits with none
        token["attributes"].append(getAttribute("Extra", i["several"]))
    if getAttribute("Hand", i["hand"])["value"] !=  "None": #for traits with none
        token["attributes"].append(getAttribute("Hand", i["hand"]))
    if getAttribute("Clothing", i["clothing"])["value"] !=  "None": #for traits with none
        token["attributes"].append(getAttribute("Clothes", i["clothing"]))
    #if getAttribute("Ancestry", i["ancestry"])["value"] !=  "None": #for traits with none
    #    token["attributes"].append(getAttribute("Ancestry", i["ancestry"]))
  
    with open('../XXXX/metadata/' + str(token_id) + '.json', 'w') as outfile:
        json.dump(token, outfile, indent=4)

In [44]:
ancestry

['Demigod', 'Royal']

In [45]:
ancestry_weights = [0.3, 0.7]
ancestry_weights

[0.3, 0.7]

In [ ]:
# Add other seals to the image:

gods_file = pd.read_csv("Gods.csv", header=0, usecols='ferrymenId')
    
def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
        
for i in range(0, 200):
    if i not in gods_file.values:
        ancestry_pick = random.choices(ancestry, ancestry_weights)[0]
        im1 = Image.open(f'./images/{i}.png').convert('RGBA')
        im2 = Image.open(f'./trait-layers/Ancestry/{file_names_dict[ancestry_pick]}.png').convert('RGBA')

        #Create each composite
        com1 = Image.alpha_composite(im1, im2)
    
    
        #Convert to RGB
        rgb_im = com1.convert('RGBA')
        file_name = str(i) + ".png"
        rgb_im.save("../XXXX/images/" + file_name)
        
        
# Add trait in metadata
        f = open(f'../XXXX/metadata/{i}.json',) 
        token = json.load(f)
        token["attributes"].append(getAttribute("Ancestry", ancestry_pick))
        
        with open('../XXXX/metadata/' + str(i) + '.json', 'w') as outfile:
            json.dump(token, outfile, indent=4)
        #token = {"attributes": []}



In [ ]:
# Add god seal to the image:


for i in range(len(gods_file)):
    god = gods_file.iloc[i].values[0]
    im1 = Image.open(f'./images/{god}.png').convert('RGBA')
    im2 = Image.open(f'./trait-layers/Ancestry/god.png').convert('RGBA')
    
    
     #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    
    
    #Convert to RGB
    rgb_im = com1.convert('RGBA')
    file_name = str(god) + ".png"
    rgb_im.save("../XXXX/images/" + file_name)
    
    f = open(f'../XXXX/metadata/{i}.json',) 
    token = json.load(f)
    token["attributes"].append(getAttribute("Ancestry", "God"))
        
    with open('../XXXX/metadata/' + str(i) + '.json', 'w') as outfile:
        json.dump(token, outfile, indent=4)
    #token = {"attributes": []}
    